In [21]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from languages import LANGUAGES

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
loaded_df = pd.read_csv("English_Processed.csv")

## Phonetic Inventory

In [23]:
print(len(loaded_df))

126054


In [24]:
# Extract the list of pronunciations
pronunciation_list = []
for p in loaded_df['pronunciation']:
    _pronunciations = p.split(',')
    pronunciation_list.extend(_pronunciations)

In [25]:
print(len(pronunciation_list))

126054


In [26]:
# Print a histogram of the different symbols
symbols_hist = {}

for P in pronunciation_list:
    # Make use of only the narrow transcriptions.
    for x in P:
        if x in symbols_hist.keys():
            symbols_hist[x] += 1
        else:
            symbols_hist[x] = 1  

# Sort the symbols 
symbols = sorted(symbols_hist, key = lambda x: symbols_hist[x], reverse = True)

In [27]:
for x in symbols:
    print(x, symbols_hist[x]) 

/ 243998
ˈ 108659
ə 87382
ɪ 85400
t 57628
n 56618
. 49403
s 48654
ɹ 47957
l 44831
k 41398
d 37409
ː 34861
i 33967
m 28404
ʊ 24906
ˌ 24905
p 23985
ɛ 23029
æ 21925
b 20264
a 17685
ɑ 16886
e 15881
f 14851
z 13948
ʃ 13700
ɡ 12024
ɔ 11077
u 10912
ɒ 10577
( 10408
) 10407
v 10266
o 9905
  9800
ʌ 9735
͡ 9692
j 8758
ʒ 8510
w 8090
ŋ 7695
h 6977
ɚ 6367
[ 3986
] 3986
θ 3872
- 3370
ɜ 3123
̩ 2746
ɝ 2029
r 1733
ð 1270
ɾ 1218
̯ 1148
ɫ 827
ʰ 799
̃ 434
ɐ 368
ʔ 344
ʉ 337
ɘ 262
x 250
ʍ 229
̥ 154
‿ 135
̠ 134
ɨ 132
̚ 126
˦ 113
̈ 103
˨ 100
̪ 91
ʷ 86
ä 74
˧ 73
ɵ 73
̞ 61
̊ 50
ʁ 49
̆ 46
ɬ 41
ˑ 38
˩ 36
ɱ 36
ɯ 33
˞ 31
̟ 30
y 30
~ 30
˥ 28
ç 27
ø 25
ʈ 23
ɻ 22
˔ 21
̬ 18
ʲ 18
̝ 17
ɖ 17
œ 17
ˀ 15
ʱ 15
ɕ 15
ɲ 15
ɸ 15
ʏ 13
ᵊ 13
ʋ 13
χ 12
ẽ 11
ĩ 11
‖ 11
õ 10
ˤ 9
ũ 9
̰ 9
ɦ 9
ʼ 8
c 8
| 8
ã 8
ɣ 6
ǀ 6
ⁿ 5
̍ 5
² 5
ɳ 5
β 5
ŏ 5
q 5
ɞ 4
ˠ 4
ɤ 4
ɟ 4
ᵈ 4
⁵ 4
ǃ 3
͜ 3
ʙ 3
ħ 3
ă 3
ɭ 3
̀ 3
ʀ 3
ʎ 2
↓ 2
¹ 2
̙ 2
ï 2
᷈ 2
ā 2
á 2
ʟ 2
' 2
ö 2
ʘ 2
ˡ 2
ᵻ 2
ǁ 2
ĭ 2
ŭ 2
ɥ 2
ɴ 2
ɓ 1
̜ 1
˗ 1
͇ 1
ü 1
ʝ 1
³ 1
⁰ 1
ʐ 1
ḿ 1
↗ 1
︎ 1
ʬ 1
… 1
́ 1
ì 1
ó 1

In [28]:
for s in symbols:
    if s == "/" or s == "[" or s == "]":
        continue 
    print(s)

ˈ
ə
ɪ
t
n
.
s
ɹ
l
k
d
ː
i
m
ʊ
ˌ
p
ɛ
æ
b
a
ɑ
e
f
z
ʃ
ɡ
ɔ
u
ɒ
(
)
v
o
 
ʌ
͡
j
ʒ
w
ŋ
h
ɚ
θ
-
ɜ
̩
ɝ
r
ð
ɾ
̯
ɫ
ʰ
̃
ɐ
ʔ
ʉ
ɘ
x
ʍ
̥
‿
̠
ɨ
̚
˦
̈
˨
̪
ʷ
ä
˧
ɵ
̞
̊
ʁ
̆
ɬ
ˑ
˩
ɱ
ɯ
˞
̟
y
~
˥
ç
ø
ʈ
ɻ
˔
̬
ʲ
̝
ɖ
œ
ˀ
ʱ
ɕ
ɲ
ɸ
ʏ
ᵊ
ʋ
χ
ẽ
ĩ
‖
õ
ˤ
ũ
̰
ɦ
ʼ
c
|
ã
ɣ
ǀ
ⁿ
̍
²
ɳ
β
ŏ
q
ɞ
ˠ
ɤ
ɟ
ᵈ
⁵
ǃ
͜
ʙ
ħ
ă
ɭ
̀
ʀ
ʎ
↓
¹
̙
ï
᷈
ā
á
ʟ
'
ö
ʘ
ˡ
ᵻ
ǁ
ĭ
ŭ
ɥ
ɴ
ɓ
̜
˗
͇
ü
ʝ
³
⁰
ʐ
ḿ
↗
︎
ʬ
…
́
ì
ó
̘
ō
*
ʂ
⁴
ɽ
̽
à
ò
ʩ
𐞪


In [29]:
print(len(symbols))

192


# Tokenizer

In [30]:
from tokenizers import Tokenizer, models, processors, decoders
import pandas as pd 
import numpy as np
from languages import LANGUAGES

In [31]:
loaded_df = pd.read_csv("English_Processed.csv")
symbols = np.load("symbols.npy")
loaded_df.head()

,Unnamed: 0,pronunciation,word,language
0,0,/pəˈsɛnt/,%,English
1,1,/pɚˈsɛnt/,%,English
2,2,/kʊŋ/,!kung,English
3,3,/ǃkʊŋ/,!kung,English
4,4,/ænd lɪt/,&lit,English


In [32]:
from solution import CharacterTokenizer, PhoneticTransformer, generate_square_subsequent_mask, PhoneticsDataset
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [33]:
pronunciation_tokenizer = CharacterTokenizer(symbols, 100)
vocab = pronunciation_tokenizer.get_vocab()
phonetics_model = PhoneticTransformer(len(vocab))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, vocab)

phonetics_model = phonetics_model.to(device)

cuda {'[CLS]': 0, '[SEP]': 1, '[BOS]': 2, '[MASK]': 3, '[PAD]': 4, '[RESERVED]': 5, '[UNK]': 6, '/': 7, '.': 8, 'ˈ': 9, 't': 10, 'a': 11, 'i': 12, 's': 13, '[': 14, ']': 15, 'n': 16, 'k': 17, 'e': 18, 'o': 19, 'ː': 20, 'l': 21, 'u': 22, 'm': 23, 'r': 24, 'p': 25, ' ': 26, 'ə': 27, 'd': 28, 'ɪ': 29, 'ʲ': 30, 'ɛ': 31, 'j': 32, '͡': 33, 'ɑ': 34, '⁵': 35, 'ɔ': 36, 'ɐ': 37, '̯': 38, 'b': 39, '²': 40, 'ɾ': 41, '³': 42, '¹': 43, 'v': 44, 'ˌ': 45, '̞': 46, '̪': 47, 'ʁ': 48, 'ɡ': 49, 'z': 50, '⁴': 51, 'f': 52, 'ʃ': 53, 'ʊ': 54, 'ŋ': 55, '⁻': 56, '̃': 57, 'ʰ': 58, 'ɨ': 59, 'ɕ': 60, '̠': 61, 'w': 62, 'h': 63, '̝': 64, ')': 65, '(': 66, 'ä': 67, 'y': 68, 'ʂ': 69, '˧': 70, 'ʒ': 71, 'æ': 72, 'x': 73, 'ɫ': 74, '̚': 75, 'ʔ': 76, 'ʋ': 77, '˨': 78, '˩': 79, 'ɲ': 80, 'ð': 81, '̟': 82, '˦': 83, 'ẽ': 84, 'ʐ': 85, 'ɒ': 86, 'ɹ': 87, 'θ': 88, '̩': 89, 'ø': 90, 'ĩ': 91, '˥': 92, 'ʈ': 93, 'ɦ': 94, 'ɣ': 95, 'ɯ': 96, '⁽': 97, '⁾': 98, 'ᵝ': 99, 'β': 100, 'ɤ': 101, 'õ': 102, 'ã': 103, 'ˣ': 104, 'ʎ': 105, 'ʌ': 106, 

c:\Users\Joaquin\miniconda3\envs\agi\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
dataset = []
for i in range(0, len(loaded_df)):
    x = loaded_df.iloc[i]
    tok = pronunciation_tokenizer.encode(x["pronunciation"])
    row = []
    for t in tok: 
        row.append(t)
        # dataset.append(row.copy())      
    dataset.append(tok)
    
phonetics_dataset = PhoneticsDataset(dataset, vocab, 100)

BATCH_SIZE = 1024
dataloader = DataLoader(phonetics_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [42]:
optimizer = optim.Adam(phonetics_model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss(ignore_index=vocab.get("[PAD]"))
EPOCHS = 20

for epoch in range(EPOCHS):
    phonetics_model.train()
    total_loss = 0

    for i, (input_ids, target_ids) in tqdm(enumerate(dataloader), total=len(dataloader)):
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)
        optimizer.zero_grad()
        tgt_mask = generate_square_subsequent_mask(input_ids.size(1)).to(device)

        output = phonetics_model(input_ids, tgt_mask = tgt_mask)
        loss = criterion(output.view(-1, len(vocab)), target_ids.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

100%|██████████| 124/124 [00:45<00:00,  2.75it/s]


Epoch 1 Loss: 2.350490323958858


100%|██████████| 124/124 [00:44<00:00,  2.77it/s]


Epoch 2 Loss: 2.2967006218048835


100%|██████████| 124/124 [00:45<00:00,  2.75it/s]


...

Epoch 18 Loss: 1.8604084003356196


100%|██████████| 124/124 [00:44<00:00,  2.78it/s]


Epoch 19 Loss: 1.8351659409461483


100%|██████████| 124/124 [00:45<00:00,  2.74it/s]

Epoch 20 Loss: 1.8091837004307778


In [43]:
def generate_text(model, max_length = 100, start_token = "[CLS]", start_seq = [], end_token = "[EOS]", temperature = 2.0, p = 0.9):
    generated_sequence = [start_token]
    generated_sequence.extend(start_seq)
    for _ in range(max_length - len(generated_sequence)):
        input_ids = torch.tensor([generated_sequence], dtype=torch.long).to(device)
        tgt_mask = generate_square_subsequent_mask(input_ids.size(1)).to(device)

        with torch.no_grad():
            output = model(input_ids, tgt_mask=tgt_mask)

        next_token_logits = output[0, len(generated_sequence ) - 1, :] / temperature
        probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cum_prob = torch.cumsum(sorted_probs, dim=-1)
        cutoff_index = torch.searchsorted(cum_prob, p)

        sorted_probs = sorted_probs[:cutoff_index + 1]
        sorted_indices = sorted_indices[:cutoff_index + 1]

        next_token_id = sorted_indices[torch.multinomial(sorted_probs, num_samples=1)].item()
        generated_sequence.append(next_token_id)

        if next_token_id == end_token:
            break 

    return generated_sequence

In [47]:
for _ in range(0, 10):
    test = generate_text(phonetics_model, start_token=0, end_token=vocab["[SEP]"], temperature = 1.2, p = 0.8)
    print(pronunciation_tokenizer.decode(test[1:-1]))



ː(ːˌˈ.i.iuː.buːˈn.ə..əʊə.ˈs..əˈdəˌˈ.kˈbɛ.p.ə.d.buː.l..n.n.mɛˈdʌə.bɹəuː.ː..n.ɹ..plʌəb.buː.m)ˈbɔbɑː.
ɪˈˈˈənɒsnnˌɪɪˈn.ɪˈsə.ˈt.k.t(ʊn.ə(ɪnˈn.n(ɪsəsdəːnɪˈp.ə(ɹɪ.ɪmə.ə.ˈsnɪ.ˈn.ˈ..s.t.ɪn)s.əʊˈm ˈn.maɪəˈ

əɹtːdɹːbˈkɹɪəə(ɹiːˈˌɹiːˈdjɹ)ʊ(ʊːɹɔːɹːɹ)əʊə)ɹː(ɹ)ˈ
ɛŋŋɛʌsɛ.tɛsttɪstɪssstɪd.daɪntænssɪ.ttɪtʊteɪsnnsˈtˈmɪktɪsp.sˈpd͡sˌts.tɛnt.tə.sɪnt ˈdssˈsˈn ətɪntppˌ


ŋ.ʌŋ.ə.s..ə.n.ə..ˈɪd..d....n.s.kɑː.əˈdɪˈm.ə.ə.ˈmæs.m.s.nə.ˈ.t.n.əˈˈp..d͡ʃən.stɪə.ˈb(æ.mʌəˈb.t.ə.dɪ
